In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import joblib
import shap
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.utils import resample
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score, f1_score
from sklearn.utils import resample
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, f1_score


In [ ]:
# Load ML results

subject_Level = False # False for Trial Level, True for Subject Level

# Path To CSV file with results
results_df = pd.read_csv(r"Y:\LabMembers\Ameen\VScode Scripts\Spinal Stim Aim 1 Codes\ML Results\BRFC_preOnly_Final_10_01_25.csv")

if subject_Level:
    # For Subject Level, keep only the trial with the highest probability away from 0.5 for each subject and outcome
    results_df['prob_dist_from_0.5'] = (results_df['y_prob'] - 0.5).abs()
    indices = results_df.groupby(['Subject', 'Outcome'])['prob_dist_from_0.5'].idxmax()
    results_df = results_df.loc[indices].reset_index(drop=True)






In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score, f1_score
from sklearn.utils import resample

# Parameters
PLOT_CI = True
N_BOOTSTRAPS = 1000
seed = 123
rng = np.random.RandomState(seed)

# Plot
plt.figure(figsize=(6.5, 5))
plt.rcParams.update({
    "font.size": 11,
    "axes.labelsize": 12,
    "axes.titlesize": 13,
    "legend.fontsize": 10,
    "xtick.labelsize": 11,
    "ytick.labelsize": 11,
    "lines.linewidth": 2,
    "axes.linewidth": 1,
    "xtick.direction": "in",
    "ytick.direction": "in"
})

outcomes = results_df["Outcome"].unique()
fpr_grid = np.linspace(0, 1, 1000)
colors = plt.cm.tab10.colors  # Distinct colors

# Initialize before the loop
ci_lower_f1, ci_upper_f1 = {}, {}

for idx, outcome in enumerate(outcomes):
    df_sub = results_df[results_df["Outcome"] == outcome].copy()
    if len(df_sub["y_true"].unique()) < 2:
        print(f"Skipping ROC for '{outcome}' (only one class).")
        continue

    # True ROC
    fpr, tpr, _ = roc_curve(df_sub["y_true"], df_sub["y_prob"])
    auc_val = roc_auc_score(df_sub["y_true"], df_sub["y_prob"])
    color = colors[idx % len(colors)]

    # Bootstrap arrays
    bootstrapped_tprs = []
    bootstrapped_auc = []
    bootstrapped_f1 = []

    # Stratified bootstrap by class
    class_0 = df_sub[df_sub["y_true"] == 0]
    class_1 = df_sub[df_sub["y_true"] == 1]
    n_0, n_1 = len(class_0), len(class_1)

    if n_0 == 0 or n_1 == 0:
        print(f"Skipping outcome '{outcome}' (empty class).")
        continue

    for _ in range(N_BOOTSTRAPS):
        sample_0 = resample(class_0, replace=True, n_samples=n_0, random_state=rng)
        sample_1 = resample(class_1, replace=True, n_samples=n_1, random_state=rng)
        sample = pd.concat([sample_0, sample_1], ignore_index=True)

        if sample["y_true"].nunique() < 2:
            continue

        # ROC + AUC
        fpr_b, tpr_b, _ = roc_curve(sample["y_true"], sample["y_prob"])
        auc_b = roc_auc_score(sample["y_true"], sample["y_prob"])
        bootstrapped_auc.append(auc_b)

        # Macro F1
        f1_b = f1_score(sample["y_true"], sample["y_pred"], average="macro", zero_division=0)
        bootstrapped_f1.append(f1_b)

        if PLOT_CI:
            tpr_interp = np.interp(fpr_grid, fpr_b, tpr_b)
            tpr_interp[0] = 0.0
            tpr_interp[-1] = 1.0
            bootstrapped_tprs.append(tpr_interp)

    # Convert to arrays
    bootstrapped_auc = np.array(bootstrapped_auc)
    bootstrapped_f1 = np.array(bootstrapped_f1)

    # Percentile CIs
    ci_lower_auc, ci_upper_auc = np.percentile(bootstrapped_auc, [2.5, 97.5])
    ci_lower_f1[outcome], ci_upper_f1[outcome] = np.percentile(bootstrapped_f1, [2.5, 97.5])

    print(f"\nFor outcome '{outcome}':")
    print(f"Bootstrap 95% CI for AUC:     [{ci_lower_auc:.4f}, {ci_upper_auc:.4f}]")
    print(f"Bootstrap 95% CI for Macro F1:[{ci_lower_f1[outcome]:.4f}, {ci_upper_f1[outcome]:.4f}]")

    # Plot main ROC
    plt.plot(
        fpr, tpr,
        label=f"{outcome} (AUC = {auc_val:.2f}, 95% CI [{ci_lower_auc:.2f}, {ci_upper_auc:.2f}])",
        color=color
    )

    # Shaded CI region
    if PLOT_CI and bootstrapped_tprs:
        bootstrapped_tprs = np.array(bootstrapped_tprs)
        tpr_lower = np.percentile(bootstrapped_tprs, 2.5, axis=0)
        tpr_upper = np.percentile(bootstrapped_tprs, 97.5, axis=0)
        plt.fill_between(
            fpr_grid, tpr_lower, tpr_upper,
            alpha=0.2, color=color, linewidth=0
        )

# Plot chance line
plt.plot([0, 1], [0, 1], 'k--', lw=1, label="Chance")

# Labels and layout
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curves with 95% Confidence Intervals")
plt.legend(loc="lower right", frameon=False)
plt.grid(True, linestyle='--', alpha=0.3)
plt.tight_layout()

# Export
# plt.savefig("ROC_with_95CI.svg", format="svg", bbox_inches="tight")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, f1_score
from matplotlib import colors

#  95% CI values
ci_lookup = {
    "Frequency Label": (ci_lower_f1['Frequency Label'], ci_upper_f1['Frequency Label']),
    "Intensity Label": (ci_lower_f1['Intensity Label'], ci_upper_f1['Intensity Label'])
}

unique_outcomes = results_df['Outcome'].unique()
n = len(unique_outcomes)

fig, axes = plt.subplots(n, 1, figsize=(7, 6 * n))

if n == 1:
    axes = [axes]

# Precompute confusion matrices + F1
all_cm = []
for outcome in unique_outcomes:
    
    ci_lookup = {
    "Frequency Label": (0.73, 0.92),
    "Intensity Label": (0.70, 0.90)
}
    subset = results_df[results_df['Outcome'] == outcome]
    conf_mat = confusion_matrix(subset['y_true'], subset['y_pred'])
    row_sums = conf_mat.sum(axis=1, keepdims=True)
    cm_norm = conf_mat.astype('float') / row_sums
    cm_norm = np.nan_to_num(cm_norm)

    # Macro F1
    f1 = f1_score(subset['y_true'], subset['y_pred'], average="macro")

    # Get CI if available
    ci = ci_lookup.get(outcome, (np.nan, np.nan))

    all_cm.append((outcome, conf_mat, row_sums, cm_norm, f1, ci))

# color scale
vmin, vmax = 0.0, 1.0

# Custom orange colormap
tab_orange = colors.LinearSegmentedColormap.from_list(
    "tab_orange", ["white", "#ff7f0e"]
)

# Plot each confusion matrix
for ax, (outcome, conf_mat, row_sums, cm_norm, f1, ci) in zip(axes, all_cm):
    if outcome == "Frequency Label":
        cmap = plt.cm.Blues
    elif outcome == "Intensity Label":
        cmap = tab_orange
    else:
        cmap = plt.cm.Greys
    
    im = ax.imshow(cm_norm, interpolation='nearest', cmap=cmap,
                   vmin=vmin, vmax=vmax)
    ax.set_title(f"Confusion Matrix: {outcome}", fontsize=16)
    
    tick_marks = np.arange(conf_mat.shape[0])
    if outcome == "Frequency Label":
        custom_labels = {0: "50 Hz", 1: "30 Hz"}
    elif outcome == "Intensity Label":
        custom_labels = {0: "LOW", 1: "HIGH"}
    else:
        custom_labels = {0: "0", 1: "1"}
    
    labels = [custom_labels.get(i, i) for i in tick_marks]
    ax.set_xticks(tick_marks)
    ax.set_xticklabels(labels, fontsize=14)
    ax.set_yticks(tick_marks)
    ax.set_yticklabels(labels, fontsize=14)
    
    # Annotate each cell
    thresh = cm_norm.max() / 2.
    for i in range(conf_mat.shape[0]):
        for j in range(conf_mat.shape[1]):
            percent = cm_norm[i, j] * 100
            count = conf_mat[i, j]
            denom = row_sums[i, 0] if row_sums[i, 0] > 0 else 1
            text = f"{percent:.1f}%\n({count}/{denom})"
            ax.text(j, i, text,
                    ha="center", va="center",
                    color="white" if cm_norm[i, j] > thresh else "black",
                    fontsize=13)
    
    ax.set_ylabel("Actual", fontsize=14)
    ax.set_xlabel("Predicted", fontsize=14)
    
    # --- Add Macro F1 with CI in requested format ---
    ci_lower, ci_upper = ci
    if not np.isnan(ci_lower):
        text_str = f"Macro F1: {f1:.2f}, 95% CI [{ci_lower:.2f}, {ci_upper:.2f}]"
    else:
        text_str = f"Macro F1: {f1:.2f}"
    
    ax.text(
        -0.05, -0.25, text_str,
        ha="left", va="center", fontsize=14, weight="bold",
        transform=ax.transAxes,
        bbox=dict(boxstyle="round,pad=0.3", edgecolor="black", facecolor="white")
    )

    # Individual colorbar
    cbar = fig.colorbar(im, ax=ax, orientation='vertical',
                        fraction=0.046, pad=0.04)
    cbar.ax.tick_params(labelsize=10)

plt.subplots_adjust(hspace=0.5)
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, f1_score

# Load results
results_file = r"C:\Users\akishta03\Documents\SS_ML\ML Results\EasyEnsemble_preOnly_20250908_Final_LearningCurve.csv"
df = pd.read_csv(results_file)

# Learning Curve Plot 
outcomes = df["Outcome"].unique()
metrics = ["AUC", "Macro F1"]

fig, axes = plt.subplots(1, len(outcomes), figsize=(6 * len(outcomes), 5), sharey=True)

if len(outcomes) == 1:
    axes = [axes]  # make iterable

for ax, outcome in zip(axes, outcomes):
    df_out = df[df["Outcome"] == outcome]

    # Collect metrics per repeat × subjects
    records = []
    for (num_subj, rep), g in df_out.groupby(["Num_Subjects", "Repeat"]):
        try:
            auc = roc_auc_score(g["y_true"], g["y_prob"])
        except ValueError:
            auc = np.nan
        f1 = f1_score(g["y_true"], g["y_pred"], average="macro", zero_division=0)
        records.append({"Num_Subjects": num_subj, "Repeat": rep, "AUC": auc, "Macro F1": f1})
    
    df_scores = pd.DataFrame(records)

    # Average across repeats (no std)
    avg_scores = df_scores.groupby("Num_Subjects").mean().reset_index()

    # Plot AUC and F1
    for metric, color in zip(metrics, ["blue", "green"]):
        x_vals = avg_scores["Num_Subjects"]
        y_vals = avg_scores[metric]
        ax.plot(x_vals, y_vals, marker="o", color=color, label=metric)

    ax.set_title(f"Learning Curve: {outcome}", fontsize=13)
    ax.set_xlabel("Number of Subjects", fontsize=12)
    ax.set_ylabel("Score", fontsize=12)
    ax.legend()
    ax.grid(True, linestyle="--", alpha=0.5)

plt.tight_layout()
plt.show()
